# Backpropagation algorithm

In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from IPython.display import display
import datetime as dt 
import autograd.numpy as anp
from autograd import grad
import random
from back_prop_utils import loss, grid_search

c = 299792458  # Speed of light in m/s

In [2]:
# Use definition as in the Extractor class

def H_th_function(n, w, length):
    '''
    Inputs
    ------
    n: refractive index
    w: frequency of light being propagated
    length: length of the sample we are modelling
    
    outputs
    -------
    returns: output for the transfer function

    Method
    ------
    Equation for transfer function derived from [Input source here]

    '''
    return (4 * n) / ((n + 1) ** 2) * anp.exp(-1j * (n - 1) * w * length/ c)

# Function to round to significant figures
def round_to_sig_figs(value, sig_figs):
    if value == 0:
        return 0
    return round(value, sig_figs - int(f"{value:.1e}".split('e')[1]) - 1)



In [3]:
# TODO: generate random n,k,d for backprop algo, add noise to this.

# frequency range
interp = 2**6
freqs_THz = np.linspace(0.1, 5, interp)   # 0-5 THz 
freqs = freqs_THz * 1e12
freqs_ang = freqs * 2 * np.pi

# Generate random n, k, d
n_lims = [2, 4]
k_lims = [-0.1, 0]
d_lims = [300e-6, 500e-6]

# Define significant figures for each parameter
n_sig_figs = 6
k_sig_figs = 6
d_sig_figs = 6

# Generate data point
n = random.uniform(*n_lims)
k = random.uniform(*k_lims)
d = random.uniform(*d_lims)

# Apply significant figures
n = round_to_sig_figs(n, n_sig_figs)
k = round_to_sig_figs(k, k_sig_figs)
d = round_to_sig_figs(d, d_sig_figs)

tf_values = [H_th_function((n+k*1j), f, d) for f in freqs_ang]

H_values_clean = np.abs(tf_values)
phi_values_clean = np.unwrap(np.angle(tf_values))

# add noise or actual signals
H_values = H_values_clean + np.random.normal(0, 0.005, size=len(H_values_clean))
phi_values = phi_values_clean + np.random.normal(0, 1, size=len(phi_values_clean))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 5))

axs[0].scatter(freqs_THz, phi_values, s=8, label='noisy phase values')
axs[0].scatter(freqs_THz, phi_values_clean, s=8, label='clean phase values')
axs[0].set_title('Unwrapped phase of sample values')
axs[0].set_xlabel('Frequencies [THz]')
axs[0].set_ylabel('Angle [Rad]')

axs[1].scatter(freqs_THz, H_values, s=8, label='noisy absolute values')
axs[1].scatter(freqs_THz, H_values_clean, s=8, label='clean absolute values')
axs[1].set_title('Absolute value of transfer function')
axs[1].set_xlabel('Frequencies [THz]')
axs[1].set_ylabel('|H|')

# Annotate with n, k, d values
axs[1].set_ylabel('|H|')

# Add a label at the top-left of the entire figure (outside the subplots)
fig.text(0.05, 0.99, f'n={n:.3f}, k={k:.3f}, d={1e6*d:.1f}µm', 
         verticalalignment='top', horizontalalignment='left', 
         bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', boxstyle='round,pad=0.5'))

axs[0].legend()
axs[1].legend()
plt.show()

In [ ]:
# Perform grid-search for params
grid_search(n0=2.0, k0=-0.05, d0=0.0004, H_values=H_values, phi_values=phi_values, freqs_ang=freqs_ang, H_th_function=H_th_function, loss=loss, verbose=True)

In [ ]:
# Convert experimental data to PyTorch tensors
w_tensor = torch.tensor(freqs_ang, dtype=torch.float32)
H_exp_tensor = torch.tensor(H_values, dtype=torch.float32)  # Amplitude
phi_exp_tensor = torch.tensor(phi_values, dtype=torch.float32)  # Phase

# Define PyTorch Model
class TransferFunctionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use transformed parameters to enforce constraints
        self.n_raw = torch.nn.Parameter(torch.tensor(np.log(3.0), dtype=torch.float32))  # log ensures n > 0
        self.k_raw = torch.nn.Parameter(torch.tensor(np.log(0.05), dtype=torch.float32))  # log of positive, negated later
        self.d_raw = torch.nn.Parameter(torch.tensor(np.log(400e-6), dtype=torch.float32))  # log ensures d > 0

    def forward(self, w):
        # Apply transformations to enforce constraints
        n = torch.exp(self.n_raw)  # Exp ensures n > 0
        k = -torch.exp(self.k_raw)  # -Exp ensures k < 0
        d = torch.exp(self.d_raw)  # Exp ensures d > 0
        
        n_complex = n + 1j * k
        H_th = (4 * n_complex) / ((n_complex + 1) ** 2) * torch.exp(-1j * (n_complex - 1) * w * d / c)
        return H_th

# Initialize model and optimizer
model = TransferFunctionModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Compute predicted transfer function
    H_pred = model(w_tensor)

    # Compute amplitude and phase
    H_pred_amp = torch.abs(H_pred)
    H_pred_phase = torch.angle(H_pred)

    # Unwrap the phase using NumPy
    H_pred_phase_unwrapped = np.unwrap(H_pred_phase.detach().cpu().numpy())  # Convert to NumPy, unwrap, and convert back to tensor

    # Convert the unwrapped phase back to a PyTorch tensor
    H_pred_phase_unwrapped = torch.tensor(H_pred_phase_unwrapped, dtype=torch.float32).to(H_pred.device)

    # Compute loss
    loss_amp = torch.mean((H_pred_amp - H_exp_tensor) ** 2)
    loss_phase = torch.mean((H_pred_phase_unwrapped - phi_exp_tensor) ** 2) / torch.var(phi_exp_tensor)  # Normalize phase loss


    loss = loss_amp + loss_phase

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# Optimized parameters
print(f"Optimized n: {torch.exp(model.n_raw).item()}, k: {-torch.exp(model.k_raw).item()}, d: {torch.exp(model.d_raw).item()}")

In [ ]:
## Experimental area
# Convert experimental data to PyTorch tensors
w_tensor = torch.tensor(freqs_ang, dtype=torch.float32)
H_exp_tensor = torch.tensor(H_values, dtype=torch.float32)  # Amplitude
phi_exp_tensor = torch.tensor(phi_values, dtype=torch.float32)  # Phase

# Define PyTorch Model
class TransferFunctionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Directly initialize parameters within expected range
        self.n = torch.nn.Parameter(torch.tensor(3.0, dtype=torch.float32))  # n > 0
        self.k = torch.nn.Parameter(torch.tensor(-0.05, dtype=torch.float32))  # k < 0
        self.d = torch.nn.Parameter(torch.tensor(400e-6, dtype=torch.float32))  # d > 0

    def forward(self, w):
        # Ensure parameters satisfy constraints
        n = torch.abs(self.n)  # Ensure n > 0
        k = -torch.abs(self.k)  # Ensure k < 0
        d = torch.abs(self.d)  # Ensure d > 0

        n_complex = n + 1j * k
        H_th = (4 * n_complex) / ((n_complex + 1) ** 2) * torch.exp(-1j * (n_complex - 1) * w * d / c)
        return H_th

# Initialize model and optimizer
model = TransferFunctionModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

# Training Loop
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Compute predicted transfer function
    H_pred = model(w_tensor)

    # Compute amplitude and phase
    H_pred_amp = torch.abs(H_pred)
    H_pred_phase = torch.angle(H_pred)

    # Unwrap the phase using NumPy
    H_pred_phase_unwrapped = np.unwrap(H_pred_phase.detach().cpu().numpy())  # Convert to NumPy, unwrap, and convert back to tensor

    # Convert the unwrapped phase back to a PyTorch tensor
    H_pred_phase_unwrapped = torch.tensor(H_pred_phase_unwrapped, dtype=torch.float32).to(H_pred.device)

    l = loss(H_exp_tensor, H_pred_amp, phi_exp_tensor, H_pred_phase_unwrapped)

    # Backpropagation
    l.backward()
    optimizer.step()

    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# Optimized parameters
print(f"Optimized n: {model.n.item()}, k: {model.k.item()}, d: {model.d.item()}")

In [8]:
# TODO: 
# Test the outputs by plotting.
# Test different initial conditions.
# Test single thickness back-prop.
# Create training plots.